## Portfolio Optimization: Modern Portfolio Theory.

Core Idea: MPT (Markowitz) finds the "efficient frontier"—portfolios maximizing return for given risk (std dev). Use optimization to minimize variance subject to target return, or max Sharpe ratio (return/risk).

In [ ]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def portfolio_performance(weights, returns, cov_matrix):
    """Calc portfolio return and volatility."""
    port_return = np.dot(weights, returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return port_return, port_vol

def neg_sharpe(weights, returns, cov_matrix, risk_free=0.02):
    """Negative Sharpe for minimization."""
    ret, vol = portfolio_performance(weights, returns, cov_matrix)
    return -(ret - risk_free) / vol

def optimize_portfolio(tickers, target_return=None):
    """MPT: Efficient frontier or max Sharpe."""
    data = yf.download(tickers, period='5y')['Adj Close']
    returns = data.pct_change().dropna()
    mean_returns = returns.mean() * 252  # Annualize
    cov_matrix = returns.cov() * 252
    
    num_assets = len(tickers)
    args = (mean_returns, cov_matrix)
    
    # Constraints: Weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # Bounds: 0-1 per asset (no shorts)
    bounds = tuple((0, 1) for _ in range(num_assets))
    initial_guess = num_assets * [1. / num_assets]
    
    if target_return:
        # Min vol for target return
        constraints.append({'type': 'eq', 'fun': lambda x: portfolio_performance(x, *args)[0] - target_return})
        result = minimize(lambda x: portfolio_performance(x, *args)[1], initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    else:
        # Max Sharpe
        result = minimize(neg_sharpe, initial_guess, args=(*args, 0.02), method='SLSQP', bounds=bounds, constraints=constraints)
    
    optimal_weights = result.x
    opt_return, opt_vol = portfolio_performance(optimal_weights, *args)
    sharpe = (opt_return - 0.02) / opt_vol
    
    return optimal_weights, opt_return, opt_vol, sharpe

# Example: Tech portfolio
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
weights, ret, vol, sharpe = optimize_portfolio(tickers)

print(f"Optimal Weights: {dict(zip(tickers, np.round(weights, 3)))}")
print(f"Expected Return: {ret:.1%}, Volatility: {vol:.1%}, Sharpe: {sharpe:.2f}")

# Plot frontier (optional)
def plot_frontier(tickers):
    data = yf.download(tickers, period='5y')['Adj Close']
    returns = data.pct_change().dropna()
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252
    
    num_portfolios = 10000
    results = np.zeros((3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(tickers))
        weights /= np.sum(weights)
        r, v = portfolio_performance(weights, mean_returns, cov_matrix)
        results[0, i] = r
        results[1, i] = v
        results[2, i] = (r - 0.02) / v
    
    plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='viridis')
    plt.colorbar(label='Sharpe Ratio')
    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.title('Efficient Frontier')
    plt.show()

plot_frontier(tickers)